In [8]:
import pandas as pd
import numpy as np
import matplotlib as plt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [38]:
list(df.columns)

['authority_code',
 'year',
 'auth_anno',
 'fpsb_auction',
 'n_bidders',
 'discount',
 'reserve_price',
 'work_category',
 'complex_work',
 'delay_ratio',
 'overrun_ratio',
 'days_to_award',
 'fiscal_efficiency',
 'municipality',
 'population',
 'experience',
 'pre_experience',
 'post_experience',
 'sample',
 'missing',
 'post_ref_adpt',
 'turin_co_sample',
 'turin_pr_sample',
 'turin_co_sample_full',
 'turin_pr_sample_full',
 'ctrl_pop_turin_co_sample',
 'ctrl_exp_turin_co_sample',
 'ctrl_pop_exp_turin_co_sample',
 'ctrl_pop_turin_pr_sample',
 'ctrl_exp_turin_pr_sample',
 'ctrl_pop_exp_turin_pr_sample',
 'OG03_dummy',
 'OG01_dummy',
 'OG_rest_dummy',
 'OG_dummy',
 'OS_dummy',
 'trend',
 'trend_treat',
 'trend_control',
 'id_auth']

In [6]:
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
#df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]

# 1. PanelOLS

In [4]:
!pip install linearmodels  
from linearmodels import PanelOLS
import statsmodels.api as sm

In [7]:
'''df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
authority_code = pd.Categorical(df_reg_exp.authority_code)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = authority_code
df_reg_exp.head()   
#exog_var = ['fpsb_auction','year','auth_anno']
#exog = sm.add_constant(df_reg_exp[exog_var])
#mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = False)  
#fe_res = mod.fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})  
#print(fe_res) '''

auth_anno  fpsb_auction  n_bidders   discount  \
authority_code year                                                    
3090272.0      2000.0  3090272.0           0.0       85.0  21.160000   
               2000.0  3090272.0           0.0       55.0  14.570000   
               2000.0  3090272.0           0.0       41.0  20.889999   
               2000.0  3090272.0           0.0       67.0  15.850000   
               2000.0  3090272.0           0.0       51.0  26.700001   

                       reserve_price work_category  complex_work  delay_ratio  \
authority_code year                                                             
3090272.0      2000.0       655828.0          OS24           1.0   116.867470   
               2000.0       640778.0          OG03           1.0   153.556488   
               2000.0       325368.0          OG01           2.0   206.722687   
               2000.0      1851498.0          OG03           1.0          NaN   
               2000.0       381504.0          OS21           2.0   103.355705   

                       overrun_ratio  days_to_award  ...  \
authority_code year                                  ...   
3090272.0      2000.0      11.712280          213.0  ...   
               2000.0      14.264524          216.0  ...   
               2000.0            NaN           44.0  ...   
               2000.0      11.934337           15.0  ...   
               2000.0            NaN          106.0  ...   

                       ctrl_pop_exp_turin_pr_sample  OG03_dummy  OG01_dummy  \
authority_code year                                                           
3090272.0      2000.0                           1.0           0           0   
               2000.0                           1.0           1           0   
               2000.0                           1.0           0           1   
               2000.0                           1.0           1           0   
               2000.0                           1.0           0           0   

                       OG_rest_dummy  OG_dummy  OS_dummy  trend  trend_treat  \
authority_code year                                                            
3090272.0      2000.0              1         0         1    1.0          1.0   
               2000.0              0         1         0    1.0          1.0   
               2000.0              0         1         0    1.0          1.0   
               2000.0              0         1         0    1.0          1.0   
               2000.0              1         0         1    1.0          1.0   

                       trend_control    year  
authority_code year                           
3090272.0      2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  
               2000.0            0.0  2000.0  

[5 rows x 38 columns]

In [20]:
#table2 = co
#outcomes "discount delay_ratio overrun_ratio days_to_award"
#(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1) obs=1262 checked
outcomes = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

authority_code  year  
3.0             2005.0    15.200000
                2005.0    14.720000
                2002.0    17.879999
                2005.0    21.830000
                2005.0    19.180000
Name: discount, dtype: float64
authority_code  year  
3.0             2005.0    40.547947
                2005.0    17.362270
                2002.0          NaN
                2005.0     0.000000
                2005.0    38.333332
Name: delay_ratio, dtype: float32
authority_code  year  
3.0             2005.0    4.843400
                2005.0    4.794236
                2002.0   -0.004622
                2005.0   -3.869733
                2005.0    4.913357
Name: overrun_ratio, dtype: float32
authority_code  year  
3.0             2005.0     5.0
                2005.0    50.0
                2002.0     8.0
                2005.0    62.0
                2005.0    23.0
Name: days_to_award, dtype: float32


In [31]:
#Winning discount
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.discount, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:               discount   R-squared:                        0.4729
Estimator:                   PanelOLS   R-squared (Between):              0.9897
No. Observations:                1262   R-squared (Within):               0.2000
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.4674
Time:                        10:45:18   Log-likelihood                   -3953.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      74.164
Entities:                          15   P-value                           0.0000
Avg Obs:                       84.133   Distribution:                 F(15,1240)
Min Obs:                       28.000                                           
Max Obs:                       200.00   F-statistic (robust):             74.164
                            

In [32]:
#completion time dealy
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.delay_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:            delay_ratio   R-squared:                        0.0741
Estimator:                   PanelOLS   R-squared (Between):              0.9910
No. Observations:                1110   R-squared (Within):               0.0002
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.0717
Time:                        10:45:46   Log-likelihood                   -6227.0
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      5.8070
Entities:                          15   P-value                           0.0000
Avg Obs:                       74.000   Distribution:                 F(15,1088)
Min Obs:                       21.000                                           
Max Obs:                       167.00   F-statistic (robust):             5.8070
                            

In [33]:
#cost_overrun
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
id_auth = pd.Categorical(df_reg_exp.id_auth)
df_reg_exp = df_reg_exp.set_index(['authority_code', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.overrun_ratio, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:          overrun_ratio   R-squared:                        0.0960
Estimator:                   PanelOLS   R-squared (Between):              0.9970
No. Observations:                1092   R-squared (Within):               0.0175
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.1038
Time:                        10:48:47   Log-likelihood                   -4230.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      7.5737
Entities:                          15   P-value                           0.0000
Avg Obs:                       72.800   Distribution:                 F(15,1070)
Min Obs:                       23.000                                           
Max Obs:                       162.00   F-statistic (robust):             7.5737
                            

In [39]:
#days to award the contraction
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
auth_anno = pd.Categorical(df_reg_exp.auth_anno)
df_reg_exp = df_reg_exp.set_index(['auth_anno', 'year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = id_auth
exog_var = ['fpsb_auction','year','auth_anno']
exog = sm.add_constant(df_reg_exp[exog_var])
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True,drop_absorbed = True)  
fe_res = mod.fit()
print(fe_res) 
#value chekced

                          PanelOLS Estimation Summary                           
Dep. Variable:          days_to_award   R-squared:                        0.4810
Estimator:                   PanelOLS   R-squared (Between):              0.5247
No. Observations:                 777   R-squared (Within):               0.0000
Date:                Wed, Jul 01 2020   R-squared (Overall):              0.4655
Time:                        11:00:26   Log-likelihood                   -3949.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      46.642
Entities:                          88   P-value                           0.0000
Avg Obs:                       8.8295   Distribution:                  F(15,755)
Min Obs:                       1.0000                                           
Max Obs:                       44.000   F-statistic (robust):             46.642
                            

In [ ]:
#col2
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
exog_var = ['fpsb_auction','year','work_category','auth_anno','reserve_price', 'municipality']
x = df_reg_exp[exog_var]
y = df_reg_exp['days_to_award']
x = sm.add_constant(x)
model = sm.OLS(y,x).fit(cov_type='cluster',cov_kwds = {'groups': df_reg_exp['auth_anno']})
result = model.predict(x)
model.summary()

In [ ]:
#col2
#fpsb_auction  i.year i.work_category reserve_price municipality
#days to award the contraction
df_reg_exp = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
year = pd.Categorical(df_reg_exp.year)
auth_anno = pd.Categorical(df_reg_exp.auth_anno)
#work_category = pd.Categorical(df_reg_exp.work_category)

df_reg_exp = df_reg_exp.set_index(['auth_anno','year'])
df_reg_exp['year'] = year
df_reg_exp['auth_anno'] = auth_anno
#df_reg_exp['work_category'] = work_category

exog_var = ['fpsb_auction','year','work_category','auth_anno','reserve_price', 'municipality']
exog = sm.add_constant(df_reg_exp[exog_var])
df_reg_exp.head()
mod = PanelOLS(df_reg_exp.days_to_award, exog, time_effects=True, drop_absorbed = True)  
#fe_res = mod.fit()
#print(fe_res) 
#value chekced


# working category

In [ ]:
#Iwork 1~42 pattern unfound
#G인지 S인지 판단
#G이면 sorted된 거에서 인덱스 +1 =Iwork_nm
#S이면 sorted된 거에서 index + 17

work_ca = df['work_category'].values.tolist()
work_ca = list(set(work_ca))
#print(work_ca)
#total 42

os_list = []
og_list = []
for i in range(len(work_ca)):
    if work_ca[i][1]=='G':
        work_ca[i] = int(work_ca[i][2:])
        og_list.append(work_ca[i])
    else:
        work_ca[i] = int(work_ca[i][2:])
        os_list.append(work_ca[i])

os_list = sorted(os_list)
og_list = sorted(og_list)


print(df.loc[0,'work_category'][1])

work_ca_order = og_list + os_list
#work_ca_order 대신 (1,43)

for i in range(1,43):
    name = 'Iwork_ca_'+str(i+1) 
    df[name] = 0
    for j in range(len(df)):
        if df.loc[j,'work_category'][1] == 'G':
            work_OG = int(df.loc[j, 'work_category'][2:])
            for g in range(len(og_list)):
                if work_OG == og_list[g]:
                    idx_g = str(g+1)
                    df.loc[j, 'Iwork_ca_' + idx_g] = 1
        else:
            work_OS = int(df.loc[j, 'work_category'][2:])
            for s in range(len(os_list)):
                if work_OS == os_list[s]:
                    idx_s = str(s + 17)
                    df.loc[j, 'Iwork_ca_'+ idx_s] = 1